# Frequency and time-frequency sensors analysis


The objective is to show you how to explore the spectral content
of your data (frequency and time-frequency). Here we'll work on Epochs.

    Authors: Alexandre Gramfort <alexandre.gramfort@inria.fr>
             Stefan Appelhoff <stefan.appelhoff@mailbox.org>
             Richard Höchenberger <richard.hoechenberger@gmail.com>
             Denis A. Engemann <denis.engemann@gmail.com>
             Richard Höchenberger <richard.hoechenberger@gmail.com>
    License: BSD (3-clause)

In [ ]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

Set parameters



In [ ]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')

In [ ]:
epochs.apply_proj()
epochs_auditory = epochs['Auditory']

Frequency analysis
------------------

We start by exploring the frequence content of our epochs.



Let's first check out all channel types by averaging across epochs.



In [ ]:
epochs_auditory.plot_psd(fmin=2., fmax=40., average=True, bandwidth=2)

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li> Select a frequency range in the plot to inspect topographies</li>

<li> The "bandwidth" parameter controls the spectral resolution of the multitaper. You can increase the resolution by chosing a narrower bandwidth at the cost of longer computation time.</li>
    </ul>
</div>

Now let's take a look at the spatial distributions of the PSD.



In [ ]:
epochs_auditory.plot_psd_topomap(ch_type='eeg', normalize=False)

In [ ]:
epochs_auditory.plot_psd_topomap(ch_type='mag', normalize=False)

In [ ]:
epochs_auditory.plot_psd_topomap(ch_type='grad', normalize=False)

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>Sometimes it can be interesting  to consider the relative power, defined as the power in a given band divided by the total power. To explore this option, have a look at the "normalize" keyword. </li>
    </ul>
</div>

## Time-frequency analysis: power and inter-trial coherence

We now compute time-frequency representations (TFRs) from our Epochs.
We'll look at power and inter-trial coherence (ITC).

To this we'll use the function `mne.time_frequency.tfr_morlet`
but you can also use `mne.time_frequency.tfr_multitaper`
or `mne.time_frequency.tfr_stockwell`.

In [ ]:
import numpy as np

# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([2, 30]), num=20)
n_cycles = freqs / 2.  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(epochs_auditory, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)

In [ ]:
power.crop(-0.1, 0.7)  # crop to remove edge artifacts

In [ ]:
itc.crop(-0.1, 0.7)  # crop to remove edge artifacts

Inspect power
-------------

<div class="alert alert-info"><h4>Note</h4><p>The generated figures are interactive. In the topo you can click
    on an image to visualize the data for one sensor.
    You can also select a portion in the time-frequency plane to
    obtain a topomap for a certain time-frequency region.</p></div>



In [ ]:
baseline_mode = 'logratio'
baseline = (None, 0)

## Plot power topomap

In [ ]:
(power.copy()
 .pick_types(eeg=True, meg=False)
 .plot_topo())

## Plot power of an individual channel

In [ ]:
power.plot(picks='EEG 050', baseline=baseline, mode=baseline_mode)

## Plot topomaps for specified frequency ranges

In [ ]:
import matplotlib.pyplot as plt

fig, axis = plt.subplots(1, 3, figsize=(7, 4))
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=4, fmax=7,
                   baseline=baseline, mode=baseline_mode, axes=axis[0],
                   title='Theta', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=8, fmax=12,
                   baseline=baseline, mode=baseline_mode, axes=axis[1],
                   title='Alpha', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=15, fmax=30,
                   baseline=baseline, mode=baseline_mode, axes=axis[2],
                   title='Beta', show=False, contours=1)
mne.viz.tight_layout()
plt.show()

Joint Plot
----------
You can also create a joint plot showing both the aggregated TFR
across channels and topomaps at specific times and frequencies to obtain
a quick overview regarding oscillatory effects across time and space.



In [ ]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.05, 2.), (0.1, 11.)])
plt.show()

Inspect ITC
-----------



In [ ]:
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=0.5, cmap='Reds')

<div class="alert alert-info"><h4>Note</h4><p>Baseline correction can be applied to power or done in plots.
    To illustrate the baseline correction in plots, the next line is
    commented power.apply_baseline(baseline=(-0.5, 0), mode='logratio')</p></div>

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Visualize the inter-trial coherence values as topomaps as done with
     power</li>
    </ul>
</div>